## 1. 환경 설정

`(1) LangSmith 설정 확인`
- .env 파일에 아래 내용을 반영
    - LANGCHAIN_TRACING_V2=true  
    - LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"  
    - LANGCHAIN_API_KEY="인증키를 입력하세요"  
    - LANGCHAIN_PROJECT="프로젝트명"  

`(2) 기본 라이브러리`

In [1]:
import os
from glob import glob

from pprint import pprint
import json

import numpy as np
import pandas as pd

`(3) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# Langsmith tracing 여부를 확인 (true: langsmith 추적 활성화, false: langsmith 추적 비활성화)
print("langsmith 추적 여부: ", os.getenv('LANGCHAIN_TRACING_V2'))

## 2.  벡터저장소 로드

In [ ]:
# 벡터스토어 로드
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

chroma_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="hf_bge_m3",
    persist_directory="./chroma_db",
)

In [ ]:
# 기본 retriever 초기화
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2}
)

query = "리비안의 성장 동력은 무엇인가요?"
retrieved_docs = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

## 3. 고급 검색기법

### 3-1. 쿼리(Query) 확장

`(1) Multi Query`
1. Retriever에 쿼리를 생성할 LLM을 지정
1. LLM이 다양한 관점에서 여러 개의 쿼리를 생성

- MultiQueryRetriever 활용

In [ ]:
# 멀티 쿼리 생성
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.7,
    max_tokens=100,
)


# 기본 retriever를 이용한 멀티 쿼리 생성 
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=chroma_k_retriever, llm=llm
)

query = "리비안의 성장 동력은 무엇인가요?"
retrieved_docs = multi_query_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()


- Custom Prompt 활용  

In [ ]:
from typing import List

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 출력 파서: LLM 결과를 질문 리스트로 변환
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""

    def parse(self, text: str) -> List[str]:
        """Split the text into lines and remove empty lines."""
        return [line.strip() for line in text.strip().split("\n") if line.strip()]
    

# 쿼리 생성 프롬프트
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Generate three different versions of the given user question to retrieve relevant documents from a vector database. The goal is to reframe the question from various perspectives to overcome limitations of distance-based similarity search.

    The generated questions should have the following characteristics:
    1. Maintain the core intent of the original question but use different expressions or viewpoints.
    2. Include synonyms or related concepts where possible.
    3. Slightly broaden or narrow the scope of the question to potentially include diverse relevant information.

    Write each question on a new line and include only the questions.

    [Original question]
    {question}
    
    [Alternative questions]
    """,
)


# 멀티쿼리 체인 구성
multiquery_chain = QUERY_PROMPT | llm | LineListOutputParser()

# 테스트 쿼리 실행
query = "리비안의 성장 동력은 무엇인가요?"
result = multiquery_chain.invoke({"question": query})

print("생성된 대안 질문들:")
for i, q in enumerate(result, 1):
    print(f"{i}. {q}")

In [ ]:
# 다중 쿼리 검색기 생성
multi_query_custom_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever, # 기본 retriever
    llm_chain=multiquery_chain,   # 멀티쿼리 체인
    parser_key="lines"            # "lines": 출력 파서의 키
)  

retrieved_docs = multi_query_custom_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

`(2) Decomposition`
  
1. 입력된 질문을 여러 개의 하위 질문 또는 하위 문제로 분해 (LEAST-TO-MOST PROMPTING)  

In [ ]:
from langchain.prompts import PromptTemplate
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to decompose the given input question into multiple sub-questions. 
    The goal is to break down the input into a set of sub-problems/sub-questions that can be answered independently.

    Follow these guidelines to generate the sub-questions:
    1. Cover various aspects related to the core topic of the original question.
    2. Each sub-question should be specific, clear, and answerable independently.
    3. Ensure that the sub-questions collectively address all important aspects of the original question.
    4. Consider temporal aspects (past, present, future) where applicable.
    5. Formulate the questions in a direct and concise manner.

    [Input question] 
    {question}

    [Sub-questions (5)]
    """,
)

# 쿼리 생성 체인
decomposition_chain = QUERY_PROMPT | llm | LineListOutputParser()

# 테스트 쿼리 실행
query = "리비안의 성장 동력은 무엇인가요?"
result = decomposition_chain.invoke({"question": query})

print("생성된 서브 질문들:")
for i, q in enumerate(result, 1):
    print(f"{i}. {q}")

In [ ]:
# 다중 쿼리 검색기 생성
multi_query_decompostion_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever,    # 기본 retriever
    llm_chain=decomposition_chain,   # 서브 질문 생성 체인
    parser_key="lines"               # "lines": 출력 파서의 키
)  

retrieved_docs = multi_query_decompostion_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

`(3) 검색 성능 평가`

In [ ]:
# 테스트 데이터셋 로드

df_qa_test = pd.read_excel("./data/qa_test_revised.xlsx")
df_qa_test.head(2)

In [ ]:
chroma_k_retriever.search_kwargs

In [ ]:
# 평가지표 계산
from krag.utils import evaluate_retrieval_at_K

retrievers = {
    'top_k': chroma_k_retriever,
    'multy_query': multi_query_retriever,
    'multy_query_custom': multi_query_custom_retriever,
    'multi_query_decompostion': multi_query_decompostion_retriever,
}

print("k:", chroma_k_retriever.search_kwargs)

df_evaluation, df_evaluation_data = evaluate_retrieval_at_K(
    df_qa_test, 
    k=chroma_k_retriever.search_kwargs['k'],  
    retrievers=retrievers, 
    ensemble=False, 
    rouge_method='rouge2', 
    threshold=0.8)

In [ ]:
# A/B 테스트 결과 출력 
df_evaluation

In [ ]:
df_evaluation_data

In [ ]:
# A/B 테스트 결과 시각화
from krag.utils import visualize_retrieval_at_K

visualize_retrieval_at_K(df_evaluation, k=chroma_k_retriever.search_kwargs['k'])

### 3-2. Re-rank
- 재순위화는 검색 결과의 품질을 향상시키는 중요한 기법
- 사용자의 쿼리와 관련성이 큰 문서들을 추출하여 상위에 위치시키는 기법

`(1) Cross Encoder Reranker`

1. Hugging Face의 Cross-Encoder 모델을 사용하여 검색 결과를 재정렬
2. 크로스 인코더 아키텍처에서는 모델의 입력이 항상 데이터 쌍(예: 두 개의 문장 또는 문서)으로 구성되며, 이들은 인코더에 의해 공동으로 처리됨
3. 크로스 인코더는 검색 쿼리와 검색된 문서 간의 유사성을 계산  
https://www.sbert.net/examples/applications/cross-encoder/README.html

In [19]:
chroma_k_retriever.search_kwargs["k"] = 5

In [ ]:
chroma_k_retriever.search_kwargs

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
re_ranker = CrossEncoderReranker(model=model, top_n=3)

cross_encoder_reranker_retriever = ContextualCompressionRetriever(
    base_compressor=re_ranker, 
    base_retriever=chroma_k_retriever,
)

question = "테슬라 회장은 누구인가요?"

retrieved_docs = cross_encoder_reranker_retriever.invoke(question)

print(f"쿼리: {question}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

`(2) LLM Reranker`  

- LLM을 사용하여 초기 검색된 문서 중에서 쿼리와 관련된 정도에 따라 순서를 조정
- 예시: LLMListwiseRerank 등 

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMListwiseRerank
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

re_ranker = LLMListwiseRerank.from_llm(llm, top_n=3)
llm_reranker_retriever = ContextualCompressionRetriever(
    base_compressor=re_ranker, 
    base_retriever=chroma_k_retriever,
)

question = "테슬라 회장은 누구인가요?"

retrieved_docs = llm_reranker_retriever.invoke(question)

print(f"쿼리: {question}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

### 3-3. Contextual compression

- 맥락적 압축은 검색된 문서를 그대로 반환하는 대신, 주어진 쿼리의 맥락을 사용하여 압축하는 기법
- 쿼리와 관련된 정보만 반환되도록 하여 LLM 호출 비용을 줄이고 응답의 품질을 향상

- 구성:
    1. 기본 검색기(base retriever) 
    2. 문서 압축기(Document Compressor)


`(1) LLMChainFilter`  

- LLM을 사용하여 초기 검색된 문서 중 어떤 것을 필터링하고 어떤 것을 반환할지 결정
- 문서 내용을 압축하거나 변경하지 않음


In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
context_filter = LLMChainFilter.from_llm(llm)

llm_filter_compression_retriever = ContextualCompressionRetriever(
    base_compressor=context_filter,             # LLM 기반 압축기
    base_retriever=chroma_db.as_retriever(),          # 기본 검색기 
)


question = "테슬라 회장은 누구인가요?"

compressed_docs = llm_filter_compression_retriever.invoke(question)

print(f"쿼리: {question}")
print("검색 결과:")
for doc in compressed_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

`(2) LLMChainExtractor`  

- LLM을 사용하여 초기 반환된 문서를 순회하며 쿼리와 관련된 내용만 추출 요약

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

llm_extractor_compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,                                    # LLM 기반 압축기
    base_retriever=cross_encoder_reranker_retriever,               # 기본 검색기 (Re-rank)
)


question = "테슬라 회장은 누구인가요?"

compressed_docs = llm_extractor_compression_retriever.invoke(question)

print(f"쿼리: {question}")
print("검색 결과:")
for doc in compressed_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

`(3) EmbeddingsFilter`  

- 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환
- LLM을 사용하지 않는 방식 (LLM 호출보다 저렴하고 빠른 옵션)


In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings_filter = EmbeddingsFilter(embeddings=embeddings_model, similarity_threshold=0.5)

embed_filter_compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,                             # 임베딩 기반 압축기
    base_retriever=cross_encoder_reranker_retriever,               # 기본 검색기 (Re-rank)
)


question = "테슬라 회장은 누구인가요?"

compressed_docs = embed_filter_compression_retriever.invoke(question)

print(f"쿼리: {question}")
print("검색 결과:")
for doc in compressed_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()


`(4) DocumentCompressorPipeline`  

- 여러 압축기를 순차적으로 결합하는 방식
- BaseDocumentTransformers를 추가하여, 문서를 더 작은 조각으로 나누거나 중복 문서를 제거하는 등의 작업도 가능

In [ ]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter


# 중복 문서 제거
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings_model)

# 쿼리와 관련성이 높은 문서만 필터링
relevant_filter = EmbeddingsFilter(embeddings=embeddings_model, similarity_threshold=0.5)

# Re-ranking
re_ranker = LLMListwiseRerank.from_llm(llm, top_n=2)

pipeline_compressor = DocumentCompressorPipeline(
    transformers=[redundant_filter, relevant_filter, re_ranker]
)

pipeline_compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, 
    base_retriever=chroma_db.as_retriever()
)

question = "테슬라 회장은 누구인가요?"

compressed_docs = pipeline_compression_retriever.invoke(question)

print(f"쿼리: {question}")
print("검색 결과:")
for doc in compressed_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*100)
    print()

## 4. 답변 생성

In [ ]:
# Context 없이 사전학습된 상태에서 답변 생성
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm.invoke(question)

In [ ]:
# 각 쿼리에 대한 검색 결과를 한꺼번에 Context로 전달해서 답변을 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the following question based on this context. If the context is not relevant to the question, just answer with '답변에 필요한 근거를 찾지 못했습니다.'

[Context]
{context}

[Question]
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([f"{doc.page_content}" for doc in docs])

rag_chain = (
    {"context": pipeline_compression_retriever | format_docs, "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("테슬라 회장은 누구인가요?")

In [ ]:
rag_chain.invoke("리비안은 2011년에 어떤 차를 공개했나요?")